In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize
from scipy.stats import expon
from scipy.stats import uniform
from scipy.stats import multivariate_normal
from scipy.stats import norm

## simulation for geometric browning motion

In [2]:
c_1=1
c_2=1
sigma=2

In [3]:
x0=1

In [4]:
timestamps=np.array([0,1,2,3]) ## include 0

In [5]:
def simu_GBM(theta,timestamps,x0,num_sample): ## assume we can only observe the process at timestamps
    c_1=theta[0]
    c_2=theta[1]
    sigma=theta[2]
    y=np.zeros((num_sample,len(timestamps))) ## y value
    for i in range(num_sample):      ## simulate num-sample brwoning motions
        mean_temp=0
        for j in range(1,len(timestamps)):
            y[i][j]=norm.rvs(loc=mean_temp,scale=np.sqrt(timestamps[j]-timestamps[j-1]))
            mean_temp=y[i][j]
    
    for i in range(num_sample):
        y[i]=timestamps**2*c_1/2+c_2*(timestamps*(np.log(np.maximum(timestamps,0.0001))-1))-sigma**2/2*timestamps+sigma*y[i]
    y=x0*np.exp(y)
    return y

In [6]:
y=simu_GBM([c_1,c_2,sigma],timestamps,x0,5000)

## inference with log-likelihood

In [16]:
def log_likelihood(parameters,x0,timestamps,data):
    c1=parameters[0]
    c2=parameters[1]
    sigma=parameters[2]
    logl=0
    for i in range(len(data)):
        logl+=sum(-np.log(np.maximum(data[i]*sigma*np.sqrt(timestamps),0.0001))-(np.log(data[i])-np.log(x0)-c1/2*timestamps**2-\
                c2*timestamps*(np.log(np.maximum(timestamps,0.0001))-1)+sigma**2/2*timestamps)**2/2/sigma**2/np.maximum(timestamps,0.0001))
    return -logl

In [17]:
parameters=np.array([1.2,0.7,3])
res=optimize.minimize(log_likelihood, parameters,args=(x0,timestamps,y), method='SLSQP', bounds=((0.001,10),(0.001,10),(0.001,10)), options={'disp': True})

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -55884.37299383789
            Iterations: 11
            Function evaluations: 62
            Gradient evaluations: 11


In [18]:
res.x

array([0.96447079, 0.99554173, 1.98905574])